In [152]:
#TODO: filter the french database to be only words we want to use (@Elaine)
import csv
import numpy as np
import pandas as pd
import json
import random
import re
from datetime import datetime
import sys
from gensim.models.fasttext import FastText
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import datasets
from sklearn_som.som import SOM

In [4]:
data = pd.read_csv('data/csvs/morph.csv', sep=';', header=0, usecols=["GRAPHIE2","CATÉGORIE","ID1","ID2","GRAPHIE1","PHONÉTIQUE2","MODE","NOMBRE","GENRE2","TEMPS","PERSONNE"])

C:\Users\elain\AppData\Local\Temp\ipykernel_13760\1293943977.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data/csvs/morph.csv', sep=';', header=0, usecols=["GRAPHIE2","CATÉGORIE","ID1","ID2","GRAPHIE1","PHONÉTIQUE2","MODE","NOMBRE","GENRE2","TEMPS","PERSONNE"])


In [5]:
df = pd.DataFrame(data)
verbslocation = df[((df.CATÉGORIE == "Verbe") & (df.ID1))] #get only the verbs
verbslocation = verbslocation["GRAPHIE1"]
print(verbslocation)
df = df[((df.TEMPS == "present") & (df.MODE == "indicative"))] #get only the present tense marked verbs (thus only verbs) 

12             s'abader
76             abaisser
141            abalober
194          abalourdir
253          abandonner
              ...      
976044            voici
976115       yorubaïser
976180        zombifier
976358    éditorialiser
976411     égyptianiser
Name: GRAPHIE1, Length: 14762, dtype: object


In [203]:
print(len(df))
print(df.head(20))   
print(df.index)
print(verbslocation.index)
print(infs)

80325
       GRAPHIE2  ID2    NOMBRE        MODE    TEMPS      PERSONNE  \
25        abade   26  singular  indicative  present   firstPerson   
26        abade   27  singular  indicative  present   thirdPerson   
30      abadent   31    plural  indicative  present   thirdPerson   
45       abades   46  singular  indicative  present  secondPerson   
47       abadez   48    plural  indicative  present  secondPerson   
56      abadons   57    plural  indicative  present   firstPerson   
88      abaisse   93  singular  indicative  present   firstPerson   
89      abaisse   94  singular  indicative  present   thirdPerson   
92    abaissent   98    plural  indicative  present   thirdPerson   
106    abaisses  113  singular  indicative  present  secondPerson   
108    abaissez  115    plural  indicative  present  secondPerson   
117   abaissons  124    plural  indicative  present   firstPerson   
154     abalobe  156  singular  indicative  present   firstPerson   
155     abalobe  157  singul

In [13]:
#delete irrelevant columns, CATÉGORIE and GENRE2
df = df.drop(columns=['CATÉGORIE', 'GENRE2','GRAPHIE1','ID1'])
df['GRAPHIE1'] = [np.nan for _ in range(len(df))]

# assign df['GRAPHIE1'] according to the row index in df. Assign it the value of the row in verbslocation with the closest but smaller or equal index
verbslocationpivot = 0
for i in df.index:
    if verbslocationpivot < len(verbslocation) - 1 and i >= verbslocation.index[verbslocationpivot+1]:
        verbslocationpivot += 1    
    df.at[i,'GRAPHIE1'] = verbslocation.iloc[verbslocationpivot]

In [191]:
inputs = []
temp = []
currentinf = df.at[df.index[0],'GRAPHIE1']
infs = []
for idx, row in df.iterrows():
    if row['GRAPHIE1'] == currentinf:
        temp.append(row['GRAPHIE2'])
    else:
        if len(temp) == 6:
            inputs.append(temp)
            infs.append(currentinf)
        temp = []
        currentinf = row['GRAPHIE1']
        temp.append(row['GRAPHIE2'])
if len(temp) == 6:
    inputs.append(temp)
    infs.append(currentinf)
print(len(infs))
print(len(inputs))
print(inputs[:2])

12395
12395
[['abade', 'abade', 'abadent', 'abades', 'abadez', 'abadons'], ['abaisse', 'abaisse', 'abaissent', 'abaisses', 'abaissez', 'abaissons']]


In [192]:
#right align and normalize each conjugated verb to 5 final letters
for i in range(len(inputs)):
    for j in range(len(inputs[i])):
        inputs[i][j] = inputs[i][j].rjust(5, '0')
        #replace all _ with random fillers
        def repl_fun(match):
            return str(random.randint(0,9))
        inputs[i][j] = re.sub('0', repl_fun, inputs[i][j])
        inputs[i][j] = inputs[i][j][-5:]
print(inputs[:3])

[['abade', 'abade', 'adent', 'bades', 'badez', 'adons'], ['aisse', 'aisse', 'ssent', 'isses', 'issez', 'ssons'], ['alobe', 'alobe', 'obent', 'lobes', 'lobez', 'obons']]


In [204]:
#100 most common verbs sorted by frequency
commonverbs = ['être', 'avoir', 'pouvoir', 'vouloir', 'faire', 'aller', 'venir', 'devenir', 'prendre', 'comprendre', 'apprendre', 'voir', 'devoir', 'recevoir', 'croire', 'boire', 'acheter', 'vivre', 'dire', 'lire', 'écrire', 'rire', 'sourire', 'suivre', 'savoir', 'connaître', 'naître', 'mourir', 'dormir', 'ouvrir', 'couvrir', 'découvrir', 'mettre', 'permettre', 'promettre', 'manger', 'partager', 'partir', 'sortir', 'tenir', 'obtenir', 'produire', 'détruire', 'traduire', 'conduire', 'cuire', 'courir', 
'appeler', 'intéresser', 'inquiéter', 'souvenir', 'lever', 'parler', 'aimer', 'donner', 'demander', 'trouver', 'passer', 'porter', 'penser', 'écouter', 'rester', 'étudier', 'montrer', 'arriver', 'laisser', 'payer', 'jouer', 'changer', 'décider', 
'commencer', 'compter', 'expliquer', 'gagner', 'exister', 'travailler', 'rencontrer', 'chercher', 'entrer', 'préparer', 'vendre', 'attendre', 'perdre', 'entendre', 'répondre', 'rendre', 'prétendre', 'défendre', 'fondre', 'descendre', 'finir', 'réussir', 'choisir', 'agir', 'investir', 'obéir', 'grossir', 'grandir', 'vieillir', 'réfléchir']
#repeat them as log distribution from 1000 to 10
logdist = (np.log(1000)-np.log(10))/(len(commonverbs)-1)
logcommonverbsdct = {}
for i in range(len(commonverbs)):
    logcommonverbsdct[commonverbs[i]] = int(np.exp(np.log(1000)-i*logdist))
print(logcommonverbsdct)


{'être': 999, 'avoir': 954, 'pouvoir': 911, 'vouloir': 869, 'faire': 830, 'aller': 792, 'venir': 756, 'devenir': 722, 'prendre': 689, 'comprendre': 657, 'apprendre': 628, 'voir': 599, 'devoir': 572, 'recevoir': 546, 'croire': 521, 'boire': 497, 'acheter': 475, 'vivre': 453, 'dire': 432, 'lire': 413, 'écrire': 394, 'rire': 376, 'sourire': 359, 'suivre': 343, 'savoir': 327, 'connaître': 312, 'naître': 298, 'mourir': 284, 'dormir': 271, 'ouvrir': 259, 'couvrir': 247, 'découvrir': 236, 'mettre': 225, 'permettre': 215, 'promettre': 205, 'manger': 196, 'partager': 187, 'partir': 178, 'sortir': 170, 'tenir': 162, 'obtenir': 155, 'produire': 148, 'détruire': 141, 'traduire': 135, 'conduire': 129, 'cuire': 123, 'courir': 117, 'appeler': 112, 'intéresser': 107, 'inquiéter': 102, 'souvenir': 97, 'lever': 93, 'parler': 89, 'aimer': 84, 'donner': 81, 'demander': 77, 'trouver': 73, 'passer': 70, 'porter': 67, 'penser': 64, 'écouter': 61, 'rester': 58, 'étudier': 55, 'montrer': 53, 'arriver': 50, 'la

In [197]:
#add that many copies of each verb to the inputs
freqinputs = []
freqinfs = []
for i in range(len(infs)):
    if infs[i] in logcommonverbsdct:
        for j in range(logcommonverbsdct[infs[i]]):
            freqinputs.append(inputs[i])
            freqinfs.append(infs[i])
print(len(freqinputs))
print(len(freqinfs))

20401
20401


In [199]:
#randomize inputs and infinitives (no order)
infs = freqinfs + infs
inputs = freqinputs + inputs
correslst = []
for i in range(len(inputs)):
    correslst.append((infs[i], inputs[i]))

random.shuffle(correslst)
print(correslst[:2])
infs = [inf for inf, _ in correslst]
inputs = [input for _, input in correslst]
print(len(infs))

[('trouver', ['rouve', 'rouve', 'uvent', 'ouves', 'ouvez', 'uvons']), ('manger', ['mange', 'mange', 'ngent', 'geons', 'anges', 'angez'])]
32796


In [161]:
#randomize inputs and infinitives (frequent first)

freqcorreslst = []
indicesscrambled = list(range(len(freqinputs)))
random.shuffle(indicesscrambled)
for i in indicesscrambled:
    freqcorreslst.append((freqinfs[i],freqinputs[i]))

corres = {}
for i in range(len(inputs)):
    corres[infs[i]] = inputs[i]

correslst = list(corres.items())
random.shuffle(correslst)
correslst = freqcorreslst[:] + correslst[:]
print(correslst[:2])
infs = [inf for inf, _ in correslst]
print(len(infs))
inputs = [input for _, input in correslst]
print(len(inputs))

[('prendre', ['prend', 'rends', 'renez', 'nnent', 'enons', 'rends']), ('aller', ['allez', 'llons', '774va', '2vais', '69vas', '5vont'])]
32796
32796


In [143]:
with open('inputs.json', 'w') as f:
    json.dump(inputs, f, ensure_ascii=True)


In [144]:
with open('inputs.json', 'r') as f:
    new = json.load(f)
print(new)

[['nais', 'naissent', 'naissez', 'naissons', 'naît', 'nais'], ['permet', 'permets', 'permettent', 'permettez', 'permettons', 'permets'], ['a', 'ai', 'as', 'avez', 'avons', 'ont'], ['voient', 'vois', 'voit', 'voyez', 'voyons', 'vois'], ['prend', 'prends', 'prenez', 'prennent', 'prenons', 'prends'], ['comprend', 'comprends', 'comprenez', 'comprennent', 'comprenons', 'comprends'], ['suis', 'suit', 'suivent', 'suivez', 'suivons', 'suis'], ['achetez', 'achetons', 'achète', 'achète', 'achètent', 'achètes'], ['écris', 'écrit', 'écrivent', 'écrivez', 'écrivons', 'écris'], ['achetez', 'achetons', 'achète', 'achète', 'achètent', 'achètes'], ['prend', 'prends', 'prenez', 'prennent', 'prenons', 'prends'], ['devez', 'devons', 'dois', 'doit', 'doivent', 'dois'], ['bois', 'boit', 'boivent', 'buvez', 'buvons', 'bois'], ['peut', 'peuvent', 'peux', 'pouvez', 'pouvons', 'puis'], ['partage', 'partage', 'partagent', 'partageons', 'partages', 'partagez'], ['recevez', 'recevons', 'reçois', 'reçoit', 'reçoive

In [200]:
print(len(inputs))
allwords = np.concatenate([np.array(inputs).flatten(),np.array(infs)]).tolist()
print(len(allwords))


32796
229572


In [165]:

def logger(args):
    np.set_printoptions(threshold=sys.maxsize)
    labels = ['vector_size', 'window', 'min_count', 'sg', 'epochs', 'somM', 'somN', 'min_n', 'max_n', 'predictions']
    args = dict(zip(labels, args))
    argsstr = 'START RANDOMIZED trial ' + '\n'
    for label,arg in args.items():
        if label != 'predictions':
            argsstr += label + ': ' + str(arg) + ', \n'
    argsstr += 'END trial ' + '\n'
    dest = open("log.txt", "a")
    dest.write(argsstr)
    dest.close()
    np.set_printoptions(threshold=False)
    return "logged" + str(datetime.time)

In [113]:
def build_train(corpus_iterable, vector_size=10, window=4, min_count=1, sg=1, epochs=100, somM=3, somN=3, min_n=1, max_n=6):

    ftmodel = FastText(vector_size=vector_size, window=window, min_count=min_count, sg=sg, min_n=min_n, max_n=max_n)
    FR_ALPHA = ['a', 'à', 'â', 'æ', 'b', 'c', 'ç', 'd', 'e', 'é', 'è', 'ê', 'ë', 'f', 'g', 'h', 'i', 'î', 'ï', 'j', 'k', 'l', 'm', 'n', 'o', 'ô', 'œ', 'p', 'q', 'r', 's', 't', 'u', 'ù', 'û', 'ü', 'v', 'w', 'x', 'y', 'ÿ', 'z']
    print("alphabet initiated")
    ftmodel.build_vocab(corpus_iterable=corpus_iterable)
    print("vocab built")
    ftmodel.train(corpus_iterable, total_examples=len(allwords), epochs=epochs)

    vec_inputs = [[ftmodel.wv[word] for word in conjset] for conjset in inputs]
    conjugations = np.array(vec_inputs)
    conjugations = conjugations.reshape(conjugations.shape[0], -1)
    print(conjugations.shape)
    # # Extract just two features (just for ease of visualization)
    # iris_data = iris_data[:, :2]

    # Build a MxN SOM
    som = SOM(m=somM, n=somN, dim=6*vector_size, random_state=1234)

    # Fit it to the data
    som.fit(conjugations)

    # Assign each datapoint to its predicted cluster
    predictions = som.predict(conjugations)
    logger([vector_size, window, min_count, sg, epochs, somM, somN, min_n, max_n, predictions])
    return predictions, som, ftmodel

In [201]:
M = 4
N = 1
(predictions, sommodel, ftmodel) = build_train(allwords, vector_size=200, window=4, min_count=1, sg=1, epochs=100, somM=M, somN=N, min_n=1, max_n=5)
print(predictions[:100])
clusters = [[] for _ in range(M*N)]
infinitives = np.array(infs)
for i in range(len(infinitives)):
    cluster = predictions[i]
    clusters[cluster].append(infinitives[i])

alphabet initiated
vocab built
(32796, 1200)
[0 1 1 ... 1 2 0]


In [202]:
#log the printed content to log.txt as well
with open('log.txt', 'a') as f:
    for i in range(M*N):
        firstline = "cluster " + str(i) + ": " + str(set(clusters[i]))
        print(firstline)
        f.write(firstline + '\n')
        secondline = "cluster " + str(i) + ": " + str(len(set(clusters[i])))
        print(secondline)
        f.write(secondline + '\n')
f.close()



cluster 0: {'satoner', 'coréguler', 'coercer', 'dédotaliser', 'verticaliser', "s'encoubler", 'banderiller', 'rajuster', 'adopter', 'accravanter', 'étoupiller', 'tubérifier', 'paladiner', 'déflegmer', 'désancrer', 'célébrer', 'solidifier', 'embouquer', 'annihiler', 'replanquer', 'commuer', 'reborder', 'centraliser', 'suroffrir', 'consister', 'béatifier', 'désempêtrer', 'remplacer', 'raper', 'préfaner', 'photométrer', 'recomposer', 'accœurer', 'réédicter', 'litonner', 'déconcerter', 'authentiquer', 'notifier', 'hyperboliser', 'égailler', 'dépropaniser', 'préfacer', "s'entretailler", 'décider', 'arroser', 'linéariser', 'resignifier', 'mondifier', 'efflanquer', 'gargoter', "s'emparer", 'appâter', 'acoquiner', 'enjaler', 'avuer', 'rétifier', 'biber', 'effarer', 'licencier', 'dépolariser', 'assonancer', 'protéiner', 'lichoter', 'déchiffonner', 'réexposer', 'cesser', 'teiller', 'reprouver', 'noper', 'cryomasser', 'se gendarmer', 'floconner', 'décriquer', 'se piffrer', 'reterser', 'existantial